In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps

# Import the custom AnimalShelter class from the animal_shelter module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Set up MongoDB connection credentials
username = "aacuser"
password = "CS340"

# Connect to the database via the AnimalShelter class
db = AnimalShelter(username, password)

# Retrieve all documents from the database as a list of dictionaries
# Note: The read method should support returning a list and accepting a projection
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ returns the '_id' column with an 'ObjectID' type,
# which can cause issues with the dash_table component. 
# We remove it from the DataFrame.
df.drop(columns=['_id'], inplace=True)

# ## Debug (uncomment to print data for debugging)
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SimpleExample')

# Load and encode the logo image
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout of the dashboard
app.layout = html.Div([
    # Centered header with the creator's name
    html.Center(html.B(html.H1('Joaquin Esguerra Jr - CS340'))),
    
    # Centered logo image, resized to 25%
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={
        'height': '25%',
        'width': '25%',
    })),

    html.Hr(),  # Horizontal rule for visual separation

    # Container for interactive filtering options
    html.Div(
        # Radio items for filtering by animal type
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Tracking', 'value': 'Disaster'},
                {'label': 'All Animals', 'value': 'reset'}  # Default: Show all animals
            ],
            value='reset'  # Initial value
        ),
    ),

    html.Hr(),  # Horizontal rule for visual separation

    # Data table to display animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,  # Disable cell editing
        filter_action="native",  # Enable column filtering
        sort_action="native",  # Enable column sorting
        sort_mode="multi",  # Allow multi-column sorting
        column_selectable=False,  # Disable column selection
        row_selectable=False,  # Disable row selection
        row_deletable=False,  # Disable row deletion
        selected_columns=[],  # Initially, no columns are selected
        selected_rows=[],  # Initially, no rows are selected
        page_action="native",  # Enable pagination
        page_current=0,  # Start at the first page
        page_size=10,  # Display 10 rows per page
    ),

    html.Br(),  # Line break for spacing
    html.Hr(),  # Horizontal rule for visual separation

    # Container for the chart and map, displayed side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 # Container for the chart
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 # Container for the map
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    [Input('filter-type', 'value')]
)
def update_dashboard(radio_filter):
    """
    Updates the data table based on the selected filter type.

    Args:
        radio_filter (str): The selected filter type from the radio buttons.

    Returns:
        tuple: A tuple containing the updated data and columns for the data table.
    """
    global df
    # Filter data based on the selected radio button
    if radio_filter == 'Water':
        df = pd.DataFrame(list(db.read({"animal_type": "Dog", 
                                        "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland"]},
                                        "sex_upon_outcome": "Intact Female",
                                        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}})))
    elif radio_filter == 'Mountain':
        df = pd.DataFrame(list(db.read({"animal_type": "Dog",
                                        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                                        "sex_upon_outcome": "Intact Male",
                                        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}})))
    elif radio_filter == 'Disaster':
        df = pd.DataFrame(list(db.read({"animal_type": "Dog",
                                        "breed": {"$in": ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                                        "sex_upon_outcome": "Intact Male",
                                        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}})))
    else:  # 'reset' - Show all animals
        df = pd.DataFrame.from_records(db.read({}))

    # Remove '_id' column if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Prepare data and columns for the data table
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    documents = df.to_dict('records')

    return documents, columns

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """
    Updates the pie chart based on the data in the data table.

    Args:
        viewData (list): The data currently displayed in the data table.

    Returns:
        list: A list containing the dcc.Graph component with the updated pie chart.
    """
    # Create a DataFrame from the data table data
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart using Plotly Express
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed')  # Create a pie chart of animal breeds
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """
    Highlights selected columns in the data table.

    Args:
        selected_columns (list): A list of selected column IDs.

    Returns:
        list: A list of style dictionaries for conditional formatting.
    """
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    """
    Updates the map based on the selected row in the data table.

    Args:
        viewData (list): The data currently displayed in the data table.
        index (list): The index of the selected row in the data table.

    Returns:
        list: A list containing the dl.Map component with the updated marker.
    """
    if viewData is None or index is None:
        return  # Return early if no data or no row is selected

    dff = pd.DataFrame.from_dict(viewData)

    # Get the row index of the selected row
    row = index[0] if index else 0

    # Define the center of the map (Austin, TX)
    center_lat = 30.75
    center_lon = -97.48

    # Create the map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[center_lat, center_lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Add a marker for the selected animal
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Tooltip showing the animal's breed
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])  # Popup showing the animal's name
                ])
            ])
        ])
    ]

# Run the Dash app in debug mode
app.run_server(debug=True)



ModuleNotFoundError: No module named 'jupyter_dash'